# Time Series Modeling for Purchase Prediction

This notebook implements time series modeling approaches to predict purchases, leveraging temporal features and respecting the temporal nature of the data (Days 1-70).

## Key Objectives:
- Leverage temporal features (day_sin, day_cos, time_sin, time_cos)
- Implement proper time-based validation
- Optimize for F1 score
- Compare different modeling approaches suitable for time series

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import TimeSeriesSplit, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve
import lightgbm as lgb
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
np.random.seed(42)

# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries imported successfully!")

## 1. Load and Explore the Data

In [ ]:
# Load the data
df = pd.read_csv('/Users/jakobbullinger/Documents/Coding Projects/DSBA/Intro Machine Learning/kaggle_competition/data/imputed/df_imputed.csv')

print(f"Dataset shape: {df.shape}")
print(f"\nNumber of unique days: {df['Day'].nunique()}")
print(f"Day range: {df['Day'].min()} to {df['Day'].max()}")
print(f"\nPurchase rate: {df['Purchase'].mean():.2%}")
print(f"\nNumber of sessions: {df['Session_ID'].nunique()}")

# Display first few rows
df.head()

## 2. Temporal Analysis and Feature Engineering

In [ ]:
# Analyze purchase patterns over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Daily purchase rate
daily_stats = df.groupby('Day').agg({
    'Purchase': ['mean', 'sum', 'count']
}).round(3)
daily_stats.columns = ['purchase_rate', 'total_purchases', 'total_sessions']

axes[0, 0].plot(daily_stats.index, daily_stats['purchase_rate'], marker='o', markersize=3)
axes[0, 0].set_title('Daily Purchase Rate Over Time')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('Purchase Rate')
axes[0, 0].grid(True, alpha=0.3)

# Volume over time
axes[0, 1].bar(daily_stats.index, daily_stats['total_sessions'], alpha=0.7, label='Sessions')
axes[0, 1].bar(daily_stats.index, daily_stats['total_purchases'], alpha=0.7, label='Purchases')
axes[0, 1].set_title('Daily Volume')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Count')
axes[0, 1].legend()

# Weekly patterns
df['Week'] = (df['Day'] - 1) // 7 + 1
weekly_stats = df.groupby('Week')['Purchase'].mean()
axes[1, 0].bar(weekly_stats.index, weekly_stats.values)
axes[1, 0].set_title('Weekly Purchase Rate')
axes[1, 0].set_xlabel('Week')
axes[1, 0].set_ylabel('Purchase Rate')

# Campaign effect over time
campaign_daily = df.groupby(['Day', 'Campaign_Period_true'])['Purchase'].mean().unstack(fill_value=0)
axes[1, 1].plot(campaign_daily.index, campaign_daily[0], label='No Campaign', marker='o', markersize=3)
axes[1, 1].plot(campaign_daily.index, campaign_daily[1], label='Campaign', marker='s', markersize=3)
axes[1, 1].set_title('Purchase Rate by Campaign Status')
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Purchase Rate')
axes[1, 1].legend()

plt.tight_layout()
plt.show()

print("Summary Statistics by Week:")
print(df.groupby('Week')['Purchase'].agg(['mean', 'std', 'count']).round(3))

In [ ]:
# Create additional time-based features
print("Creating additional temporal features...")

# Rolling statistics (looking back only - no data leakage)
def create_lag_features(df, window_sizes=[3, 7, 14]):
    df_sorted = df.sort_values('Day').copy()
    
    for window in window_sizes:
        # Calculate rolling statistics for each day (excluding current day)
        daily_rates = df_sorted.groupby('Day')['Purchase'].mean().shift(1)
        rolling_mean = daily_rates.rolling(window=window, min_periods=1).mean()
        rolling_std = daily_rates.rolling(window=window, min_periods=1).std().fillna(0)
        
        # Map back to original dataframe
        df_sorted[f'purchase_rate_ma{window}'] = df_sorted['Day'].map(rolling_mean)
        df_sorted[f'purchase_rate_std{window}'] = df_sorted['Day'].map(rolling_std)
    
    return df_sorted

# Apply lag features
df = create_lag_features(df)

# Day of week features (cyclical)
df['day_of_week'] = df['Day'] % 7
df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

# Trend features
df['day_normalized'] = df['Day'] / df['Day'].max()  # Normalized day for trend
df['day_squared'] = df['day_normalized'] ** 2  # Quadratic trend

# Interaction between time and campaign
df['campaign_day_interaction'] = df['Campaign_Period_true'] * df['Day']
df['campaign_time_sin'] = df['Campaign_Period_true'] * df['time_sin']
df['campaign_time_cos'] = df['Campaign_Period_true'] * df['time_cos']

# Email interaction patterns over time
df['email_day_interaction'] = df['Email_Interaction'] * df['day_normalized']

# Create momentum features (purchase velocity)
daily_velocity = df.groupby('Day')['Purchase'].mean().diff().shift(1)
df['purchase_velocity'] = df['Day'].map(daily_velocity).fillna(0)

print(f"Total features after engineering: {len([col for col in df.columns if col not in ['id', 'Session_ID', 'Purchase']])}")
print("\nNew temporal features created:")
new_features = [col for col in df.columns if any(x in col for x in ['ma', 'std', 'velocity', 'interaction', 'normalized', 'squared'])]
print(new_features[:10], '...') if len(new_features) > 10 else print(new_features)

## 3. Time-Based Train-Test Split

For time series, we need to respect temporal ordering. We'll use the last 20% of days as test set.

In [ ]:
# Define split point
split_day = int(df['Day'].max() * 0.8)  # Day 56 for 70 days
print(f"Training on days 1-{split_day}, Testing on days {split_day+1}-{df['Day'].max()}")

# Create train and test sets
train_df = df[df['Day'] <= split_day].copy()
test_df = df[df['Day'] > split_day].copy()

print(f"\nTrain set: {train_df.shape[0]} samples (Days 1-{split_day})")
print(f"Test set: {test_df.shape[0]} samples (Days {split_day+1}-{df['Day'].max()})")
print(f"\nTrain purchase rate: {train_df['Purchase'].mean():.2%}")
print(f"Test purchase rate: {test_df['Purchase'].mean():.2%}")

# Prepare features and target
exclude_cols = ['id', 'Session_ID', 'Purchase', 'Day', 'Week']
feature_cols = [col for col in df.columns if col not in exclude_cols]

X_train = train_df[feature_cols]
y_train = train_df['Purchase']
X_test = test_df[feature_cols]
y_test = test_df['Purchase']

# Clean feature names for LightGBM (remove special characters)
print("\nCleaning feature names for LightGBM compatibility...")
clean_feature_names = {}
for col in X_train.columns:
    # Replace special characters with underscores
    clean_name = col.replace(':', '_').replace('[', '_').replace(']', '_').replace(',', '_').replace(' ', '_')
    clean_feature_names[col] = clean_name

# Rename columns
X_train = X_train.rename(columns=clean_feature_names)
X_test = X_test.rename(columns=clean_feature_names)

print(f"\nNumber of features: {X_train.shape[1]}")
print(f"Features shape - Train: {X_train.shape}, Test: {X_test.shape}")
print(f"Example cleaned feature names: {list(X_train.columns[:5])}...")

## 4. Model Development and Comparison

We'll compare multiple approaches:
1. LightGBM (strong baseline from previous work)
2. XGBoost with temporal features
3. Time-aware ensemble
4. Recency-weighted model

In [ ]:
# Helper function for model evaluation
def evaluate_model(model, X_train, y_train, X_test, y_test, model_name, threshold=0.5):
    """
    Evaluate model performance with focus on F1 score
    """
    # Get predictions
    y_train_pred_proba = model.predict_proba(X_train)[:, 1]
    y_test_pred_proba = model.predict_proba(X_test)[:, 1]
    
    # Apply threshold
    y_train_pred = (y_train_pred_proba >= threshold).astype(int)
    y_test_pred = (y_test_pred_proba >= threshold).astype(int)
    
    # Calculate metrics
    results = {
        'model': model_name,
        'threshold': threshold,
        'train_f1': f1_score(y_train, y_train_pred),
        'test_f1': f1_score(y_test, y_test_pred),
        'train_precision': precision_score(y_train, y_train_pred),
        'test_precision': precision_score(y_test, y_test_pred),
        'train_recall': recall_score(y_train, y_train_pred),
        'test_recall': recall_score(y_test, y_test_pred),
        'train_auc': roc_auc_score(y_train, y_train_pred_proba),
        'test_auc': roc_auc_score(y_test, y_test_pred_proba),
        'test_positive_rate': y_test_pred.mean()
    }
    
    print(f"\n{'='*50}")
    print(f"Model: {model_name}")
    print(f"{'='*50}")
    print(f"Threshold: {threshold:.2f}")
    print(f"\nF1 Score - Train: {results['train_f1']:.4f}, Test: {results['test_f1']:.4f}")
    print(f"Precision - Train: {results['train_precision']:.4f}, Test: {results['test_precision']:.4f}")
    print(f"Recall - Train: {results['train_recall']:.4f}, Test: {results['test_recall']:.4f}")
    print(f"AUC - Train: {results['train_auc']:.4f}, Test: {results['test_auc']:.4f}")
    print(f"\nTest Set Positive Rate: {results['test_positive_rate']:.2%}")
    
    return results, y_test_pred_proba

In [ ]:
# Model 1: LightGBM with temporal features
print("Training LightGBM with temporal features...")

lgb_params = {
    'objective': 'binary',
    'metric': 'binary_logloss',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_child_samples': 20,
    'random_state': 42,
    'verbosity': -1
}

# Create dataset
lgb_train = lgb.Dataset(X_train, y_train)
lgb_val = lgb.Dataset(X_test, y_test, reference=lgb_train)

# Train with early stopping
lgb_model = lgb.train(
    lgb_params,
    lgb_train,
    valid_sets=[lgb_val],
    num_boost_round=1000,
    callbacks=[lgb.early_stopping(50), lgb.log_evaluation(0)]
)

# Create sklearn-compatible wrapper for evaluation
class LGBWrapper:
    def __init__(self, model):
        self.model = model
    
    def predict_proba(self, X):
        proba = self.model.predict(X, num_iteration=self.model.best_iteration)
        return np.vstack([1 - proba, proba]).T

lgb_wrapper = LGBWrapper(lgb_model)
lgb_results, lgb_proba = evaluate_model(lgb_wrapper, X_train, y_train, X_test, y_test, "LightGBM", 0.5)

In [ ]:
# Model 2: XGBoost with sample weights (recent data weighted more)
print("\nTraining XGBoost with recency weighting...")

# Create sample weights - more recent samples get higher weight
train_weights = np.exp((train_df['Day'] - train_df['Day'].min()) / 30)  # Exponential decay
train_weights = train_weights / train_weights.mean()  # Normalize

xgb_model = xgb.XGBClassifier(
    n_estimators=300,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    eval_metric='logloss',
    early_stopping_rounds=50
)

xgb_model.fit(
    X_train, y_train,
    sample_weight=train_weights,
    eval_set=[(X_test, y_test)],
    verbose=False
)

xgb_results, xgb_proba = evaluate_model(xgb_model, X_train, y_train, X_test, y_test, "XGBoost (Weighted)", 0.5)

In [ ]:
# Model 3: LightGBM with Time Series Cross-Validation for hyperparameter tuning
print("\nTraining LightGBM with TimeSeriesSplit optimization...")

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

# Define parameter space
param_dist = {
    'num_leaves': randint(20, 50),
    'max_depth': randint(4, 10),
    'learning_rate': uniform(0.01, 0.15),
    'n_estimators': randint(100, 500),
    'min_child_samples': randint(10, 30),
    'subsample': uniform(0.6, 0.3),
    'colsample_bytree': uniform(0.6, 0.3),
    'reg_alpha': uniform(0, 1),
    'reg_lambda': uniform(0, 1)
}

# Time series split for validation
tscv = TimeSeriesSplit(n_splits=3)

# Base model
lgb_ts = lgb.LGBMClassifier(
    objective='binary',
    metric='binary_logloss',
    random_state=42,
    verbosity=-1
)

# Random search with time series cross-validation
print("Performing randomized search with TimeSeriesSplit...")
random_search = RandomizedSearchCV(
    lgb_ts,
    param_dist,
    n_iter=20,
    cv=tscv,
    scoring='f1',
    random_state=42,
    n_jobs=-1,
    verbose=0
)

random_search.fit(X_train, y_train)

print(f"Best F1 score from CV: {random_search.best_score_:.4f}")
print(f"Best parameters: {random_search.best_params_}")

lgb_ts_results, lgb_ts_proba = evaluate_model(random_search.best_estimator_, X_train, y_train, X_test, y_test, "LightGBM (TS-Optimized)", 0.5)

## 5. Threshold Optimization for F1 Score

In [ ]:
def find_optimal_threshold(y_true, y_proba, metric='f1'):
    """
    Find optimal threshold for classification
    """
    thresholds = np.arange(0.1, 0.9, 0.01)
    scores = []
    
    for threshold in thresholds:
        y_pred = (y_proba >= threshold).astype(int)
        if metric == 'f1':
            score = f1_score(y_true, y_pred)
        elif metric == 'precision':
            score = precision_score(y_true, y_pred)
        elif metric == 'recall':
            score = recall_score(y_true, y_pred)
        scores.append(score)
    
    optimal_idx = np.argmax(scores)
    return thresholds[optimal_idx], scores[optimal_idx]

# Find optimal thresholds for each model
print("Finding optimal thresholds for each model...\n")

models_proba = {
    'LightGBM': lgb_proba,
    'XGBoost (Weighted)': xgb_proba,
    'LightGBM (TS-Optimized)': lgb_ts_proba
}

optimal_thresholds = {}
for model_name, proba in models_proba.items():
    opt_threshold, opt_score = find_optimal_threshold(y_test, proba, 'f1')
    optimal_thresholds[model_name] = opt_threshold
    
    # Re-evaluate with optimal threshold
    y_pred_opt = (proba >= opt_threshold).astype(int)
    f1_opt = f1_score(y_test, y_pred_opt)
    precision_opt = precision_score(y_test, y_pred_opt)
    recall_opt = recall_score(y_test, y_pred_opt)
    positive_rate = y_pred_opt.mean()
    
    print(f"{model_name}:")
    print(f"  Optimal Threshold: {opt_threshold:.3f}")
    print(f"  F1 Score: {f1_opt:.4f}")
    print(f"  Precision: {precision_opt:.4f}")
    print(f"  Recall: {recall_opt:.4f}")
    print(f"  Positive Rate: {positive_rate:.2%}")
    print()

## 6. Feature Importance Analysis

In [ ]:
# Get feature importance from best model
feature_importance = lgb_model.feature_importance(importance_type='gain')
feature_names = X_train.columns

# Create importance dataframe
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': feature_importance
}).sort_values('importance', ascending=False)

# Separate temporal vs non-temporal features
temporal_keywords = ['day', 'time', 'ma', 'std', 'velocity', 'week', 'campaign_day', 'campaign_time', 'email_day']
importance_df['is_temporal'] = importance_df['feature'].apply(
    lambda x: any(keyword in x.lower() for keyword in temporal_keywords)
)

# Visualize top features
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Top 20 overall features
top_20 = importance_df.head(20)
colors = ['coral' if x else 'skyblue' for x in top_20['is_temporal']]
axes[0].barh(range(len(top_20)), top_20['importance'], color=colors)
axes[0].set_yticks(range(len(top_20)))
axes[0].set_yticklabels(top_20['feature'])
axes[0].set_xlabel('Importance')
axes[0].set_title('Top 20 Features (Red=Temporal, Blue=Non-temporal)')
axes[0].invert_yaxis()

# Temporal features only
temporal_features = importance_df[importance_df['is_temporal']].head(15)
axes[1].barh(range(len(temporal_features)), temporal_features['importance'], color='coral')
axes[1].set_yticks(range(len(temporal_features)))
axes[1].set_yticklabels(temporal_features['feature'])
axes[1].set_xlabel('Importance')
axes[1].set_title('Top Temporal Features')
axes[1].invert_yaxis()

plt.tight_layout()
plt.show()

print("\nTemporal Features Contribution:")
temporal_importance = importance_df[importance_df['is_temporal']]['importance'].sum()
total_importance = importance_df['importance'].sum()
print(f"Temporal features account for {temporal_importance/total_importance:.1%} of total importance")
print(f"\nTop 5 temporal features:")
for _, row in temporal_features.head(5).iterrows():
    print(f"  - {row['feature']}: {row['importance']:.0f}")

## 7. Time-Based Performance Analysis

In [ ]:
# Analyze model performance over time in test set
best_model_proba = lgb_ts_proba
best_threshold = optimal_thresholds['LightGBM (TS-Optimized)']

# Add predictions to test dataframe
test_df_analysis = test_df.copy()
test_df_analysis['predicted_proba'] = best_model_proba
test_df_analysis['predicted'] = (best_model_proba >= best_threshold).astype(int)

# Calculate daily metrics
daily_metrics = test_df_analysis.groupby('Day').apply(lambda x: pd.Series({
    'f1': f1_score(x['Purchase'], x['predicted']) if x['Purchase'].sum() > 0 else 0,
    'precision': precision_score(x['Purchase'], x['predicted'], zero_division=0),
    'recall': recall_score(x['Purchase'], x['predicted'], zero_division=0),
    'actual_rate': x['Purchase'].mean(),
    'predicted_rate': x['predicted'].mean(),
    'n_samples': len(x)
}))

# Visualize performance over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# F1 score over time
axes[0, 0].plot(daily_metrics.index, daily_metrics['f1'], marker='o', label='F1 Score')
axes[0, 0].axhline(y=daily_metrics['f1'].mean(), color='r', linestyle='--', label=f'Mean: {daily_metrics["f1"].mean():.3f}')
axes[0, 0].set_xlabel('Day')
axes[0, 0].set_ylabel('F1 Score')
axes[0, 0].set_title('Daily F1 Score in Test Period')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Precision and Recall
axes[0, 1].plot(daily_metrics.index, daily_metrics['precision'], marker='s', label='Precision')
axes[0, 1].plot(daily_metrics.index, daily_metrics['recall'], marker='^', label='Recall')
axes[0, 1].set_xlabel('Day')
axes[0, 1].set_ylabel('Score')
axes[0, 1].set_title('Daily Precision and Recall')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Actual vs Predicted rates
axes[1, 0].plot(daily_metrics.index, daily_metrics['actual_rate'], marker='o', label='Actual Rate')
axes[1, 0].plot(daily_metrics.index, daily_metrics['predicted_rate'], marker='s', label='Predicted Rate')
axes[1, 0].set_xlabel('Day')
axes[1, 0].set_ylabel('Purchase Rate')
axes[1, 0].set_title('Actual vs Predicted Purchase Rates')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Sample volume
axes[1, 1].bar(daily_metrics.index, daily_metrics['n_samples'], alpha=0.7)
axes[1, 1].set_xlabel('Day')
axes[1, 1].set_ylabel('Number of Samples')
axes[1, 1].set_title('Daily Sample Volume in Test Period')
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Test Period Performance Summary:")
print(f"Average F1 Score: {daily_metrics['f1'].mean():.4f} (±{daily_metrics['f1'].std():.4f})")
print(f"Best Day F1: {daily_metrics['f1'].max():.4f} (Day {daily_metrics['f1'].idxmax()})")
print(f"Worst Day F1: {daily_metrics['f1'].min():.4f} (Day {daily_metrics['f1'].idxmin()})")
print(f"\nModel stability (CV of F1): {daily_metrics['f1'].std() / daily_metrics['f1'].mean():.3f}")

## 8. Ensemble Approach with Time Decay

In [ ]:
# Create an ensemble that weights recent predictions more
print("Creating time-weighted ensemble...")

# Get predictions from all models
all_predictions = pd.DataFrame({
    'lgb': lgb_proba,
    'xgb': xgb_proba,
    'lgb_ts': lgb_ts_proba
})

# Simple average ensemble
ensemble_avg = all_predictions.mean(axis=1)

# Weighted ensemble (based on individual F1 scores)
weights = {
    'lgb': lgb_results['test_f1'],
    'xgb': xgb_results['test_f1'],
    'lgb_ts': lgb_ts_results['test_f1']
}
total_weight = sum(weights.values())
weights = {k: v/total_weight for k, v in weights.items()}

ensemble_weighted = (
    all_predictions['lgb'] * weights['lgb'] +
    all_predictions['xgb'] * weights['xgb'] +
    all_predictions['lgb_ts'] * weights['lgb_ts']
)

# Evaluate ensemble approaches
print("\nEnsemble Results:")
print("="*50)

for ensemble_name, ensemble_proba in [('Average Ensemble', ensemble_avg), ('Weighted Ensemble', ensemble_weighted)]:
    opt_threshold, opt_score = find_optimal_threshold(y_test, ensemble_proba, 'f1')
    y_pred = (ensemble_proba >= opt_threshold).astype(int)
    
    print(f"\n{ensemble_name}:")
    print(f"  Optimal Threshold: {opt_threshold:.3f}")
    print(f"  F1 Score: {f1_score(y_test, y_pred):.4f}")
    print(f"  Precision: {precision_score(y_test, y_pred):.4f}")
    print(f"  Recall: {recall_score(y_test, y_pred):.4f}")
    print(f"  AUC: {roc_auc_score(y_test, ensemble_proba):.4f}")
    print(f"  Positive Rate: {y_pred.mean():.2%}")

print("\nModel Weights in Weighted Ensemble:")
for model, weight in weights.items():
    print(f"  {model}: {weight:.3f}")

## 9. Final Model Selection and Summary

In [ ]:
# Compile all results
results_summary = pd.DataFrame([
    lgb_results,
    xgb_results,
    lgb_ts_results
])

# Add optimized thresholds results
for model_name in models_proba.keys():
    proba = models_proba[model_name]
    threshold = optimal_thresholds[model_name]
    y_pred = (proba >= threshold).astype(int)
    
    results_summary = pd.concat([results_summary, pd.DataFrame([{
        'model': f"{model_name} (Optimized)",
        'threshold': threshold,
        'test_f1': f1_score(y_test, y_pred),
        'test_precision': precision_score(y_test, y_pred),
        'test_recall': recall_score(y_test, y_pred),
        'test_auc': roc_auc_score(y_test, proba),
        'test_positive_rate': y_pred.mean()
    }])], ignore_index=True)

# Sort by F1 score
results_summary = results_summary.sort_values('test_f1', ascending=False)

print("\n" + "="*80)
print("FINAL RESULTS SUMMARY")
print("="*80)
print("\nAll Models Ranked by Test F1 Score:")
print(results_summary[['model', 'threshold', 'test_f1', 'test_precision', 'test_recall', 'test_positive_rate']].to_string(index=False))

# Best model
best_model_name = results_summary.iloc[0]['model']
best_f1 = results_summary.iloc[0]['test_f1']
best_threshold = results_summary.iloc[0]['threshold']
best_positive_rate = results_summary.iloc[0]['test_positive_rate']

print("\n" + "="*80)
print(f"BEST MODEL: {best_model_name}")
print("="*80)
print(f"F1 Score: {best_f1:.4f}")
print(f"Optimal Threshold: {best_threshold:.3f}")
print(f"Positive Rate: {best_positive_rate:.2%}")

# Marketing budget constraint check
print("\n" + "="*80)
print("BUSINESS CONSTRAINT CHECK")
print("="*80)
budget_reduction_target = 200 / 630  # Target: €200 from €630
print(f"Target spending: {budget_reduction_target:.1%} of original (€200/€630)")
print(f"Model targets: {best_positive_rate:.1%} of users")
if best_positive_rate <= budget_reduction_target + 0.05:  # Allow 5% margin
    print("✓ Model meets budget constraint!")
else:
    print(f"⚠ Model exceeds budget by {(best_positive_rate - budget_reduction_target)*100:.1f} percentage points")

## 10. Key Insights and Recommendations

### Time Series Modeling Insights:
1. **Temporal Features Impact**: Analyze which temporal features contributed most
2. **Model Stability**: How consistent is performance across different time periods?
3. **Recency Weighting**: Did giving more weight to recent data improve performance?

### Next Steps:
1. **Advanced Time Series Models**: Consider LSTM or Prophet for capturing complex temporal patterns
2. **Feature Engineering**: Create more sophisticated lag features or rolling statistics
3. **Online Learning**: Implement models that can adapt to changing patterns over time

In [ ]:
# Final insights
print("\nKEY FINDINGS:")
print("="*50)

# Compare temporal vs non-temporal features
temporal_cols = [col for col in X_train.columns if any(k in col.lower() for k in temporal_keywords)]
non_temporal_cols = [col for col in X_train.columns if col not in temporal_cols]

print(f"\n1. Feature Distribution:")
print(f"   - Total features: {len(X_train.columns)}")
print(f"   - Temporal features: {len(temporal_cols)} ({len(temporal_cols)/len(X_train.columns):.1%})")
print(f"   - Non-temporal features: {len(non_temporal_cols)} ({len(non_temporal_cols)/len(X_train.columns):.1%})")

print(f"\n2. Model Performance Comparison:")
baseline_f1 = lgb_results['test_f1']
ts_optimized_f1 = lgb_ts_results['test_f1']
improvement = (ts_optimized_f1 - baseline_f1) / baseline_f1 * 100
print(f"   - Baseline LightGBM F1: {baseline_f1:.4f}")
print(f"   - TS-Optimized LightGBM F1: {ts_optimized_f1:.4f}")
print(f"   - Improvement: {improvement:+.1f}%")

print(f"\n3. Temporal Stability:")
print(f"   - F1 Score Std Dev across days: {daily_metrics['f1'].std():.4f}")
print(f"   - Coefficient of Variation: {daily_metrics['f1'].std() / daily_metrics['f1'].mean():.3f}")

print(f"\n4. Business Impact:")
print(f"   - Optimal threshold found: {best_threshold:.3f}")
print(f"   - Users targeted: {best_positive_rate:.1%}")
print(f"   - Expected F1 Score: {best_f1:.4f}")

print("\n" + "="*50)
print("Time series modeling complete!")